<a href="https://colab.research.google.com/github/yasmeenAdel154/FRONT-END/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**this code from : https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.9 MB/s eta 0:00:00


Data Loading from Local CSV File

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

data = pd.read_csv("train.csv")
#pre processing
del data["id"]
del data["title"]

print ( data.head() )
print ( data["text"]  )


                                                text  \
0  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
1  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
2  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
3  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   
4  جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...   

                                          question  \
0                   - من هو جمال أحمد حمزة خاشقجي؟   
1        - متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال   
2      - في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال   
3   - في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال   
4  - كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال   

                                             answers  
0  {'text': array(['صحفي وإعلامي'], dtype=object)...  
1  {'text': array(['حمزة خاشقجي (13 أكتوبر 1958، ...  
2  {'text': array(['المدينة المنورة'], dtype=obje...  
3  {'text': array(['واشنطن بوست'], dtype=object),...  
4  {'text': array(['وُصف في الصحف وأجهزة الاعلام ...  
0      ج

In [ ]:
print("Number of question and answers: ", len(data))

Number of question and answers:  693


Building the Chatbot
The best part about using these pre-trained models is that you can load the model and its tokenizer in just two simple lines of code. 😲 Isn’t it simply wow? For tasks like text classification, we need to fine-tune BERT on our dataset. But for question answering tasks, we can even use the already trained model and get decent results even when our text is from a completely different domain. To get decent results, we are using a BERT model which is fine-tuned on the SQuAD benchmark.

For our task, we will use the BertForQuestionAnswering class from the transformers library.

In [ ]:
model = BertForQuestionAnswering.from_pretrained('asafaya/bert-base-arabic')
tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly i

Asking a Question
Let’s randomly select a question number.

In [ ]:
random_num = np.random.randint(0,len(data))
question = data["question"][random_num]
text = data["text"][random_num]

Let’s tokenize the question and text as a pair.

In [26]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))
print ( input_ids )

The input has a total of 162 tokens.
[2, 1809, 2105, 1947, 6830, 16683, 2083, 1833, 4650, 1725, 4871, 2086, 4901, 3799, 2083, 2145, 221, 3, 3799, 2083, 2145, 2105, 7806, 15748, 15070, 1726, 18099, 3799, 2083, 29605, 1021, 5386, 3137, 219, 11165, 1960, 2369, 6230, 3260, 3188, 5620, 3955, 219, 2269, 15669, 2095, 4052, 1725, 4304, 1809, 1914, 2523, 4022, 22969, 2832, 2635, 4089, 1726, 2091, 2145, 18, 2166, 17545, 4052, 7829, 4821, 1020, 6830, 7806, 1905, 3738, 3260, 3188, 5620, 3955, 24741, 2572, 1725, 22, 4301, 4221, 18, 2166, 17545, 4052, 24330, 1747, 23, 22481, 1719, 2179, 1855, 1008, 219, 2264, 15703, 1013, 1747, 22481, 8117, 1914, 7531, 16318, 1013, 4399, 219, 9698, 1936, 1013, 3749, 1747, 22481, 8117, 2364, 1914, 10922, 8845, 30068, 1006, 219, 10716, 8498, 14402, 2066, 31183, 1735, 18, 2526, 8144, 2608, 1726, 18099, 3799, 2083, 1833, 13015, 1725, 4871, 5044, 219, 3489, 2105, 7226, 2608, 1833, 17509, 2281, 14281, 5694, 12615, 7806, 1905, 4901, 3799, 2083, 7691, 2815, 13015, 1725, 524

Let’s see how many tokens this question and text pair have.

To look at what our tokenizer is doing, let’s just print out the tokens and their IDs.

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]          2
ما         1,809
هي         2,105
اخر        1,947
استضافة    6,830
لكاس      16,683
العالم     2,083
التي       1,833
تمت        4,650
في         1,725
اوروبا     4,871
قبل        2,086
بطولة      4,901
كاس        3,799
العالم     2,083
2018       2,145
؟            221
[SEP]          3
كاس        3,799
العالم     2,083
2018       2,145
هي         2,105
البطولة    7,806
الحادية   15,748
والعشرون  15,070
من         1,726
بطولات    18,099
كاس        3,799
العالم     2,083
لفر       29,605
##ق        1,021
الرجال     5,386
الوطنية    3,137
،            219
والمق     11,165
##امة      1,960
تحت        2,369
رعاية      6,230
الاتحاد    3,260
الدولي     3,188
لكرة       5,620
القدم      3,955
،            219
وقد        2,269
استضاف    15,669
##تها      2,095
روسيا      4,052
في         1,725
الفترة     4,304
ما         1,809
بين        1,914
14         2,523
يونيو      4,022
ولغ       22,969
##اية      2,832
15         2,635
يوليو      4,089
من         1,726
عام        2,0

In [23]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

SEP token index:  17
Number of tokens in segment A:  18
Number of tokens in segment B:  144
